In [ ]:
import numpy as np
import pandas as pd
import requests
import zipfile
import io
from datetime import datetime
from kafka import KafkaProducer
import time
import json
import random

In [ ]:
# Data from december 2020
r = requests.get('https://s3.amazonaws.com/tripdata/202012-citibike-tripdata.csv.zip')
  
with zipfile.ZipFile(io.BytesIO(r.content)) as ar:
    trip_data = pd.read_csv(ar.open('202012-citibike-tripdata.csv'))

In [ ]:
# Setting up the producer
producer = KafkaProducer(
  bootstrap_servers=["kafka:9092"],
  value_serializer=lambda v: json.dumps(v).encode("utf-8"))

In [ ]:
# Preprocessing 
trip_data.rename(columns = {'start station id': 'S', 
                            'end station id': 'E',
                            'birth year': 'BY',
                            'bikeid':'B',
                            'usertype':'U',
                            'gender':'G',
                            'tripduration':'D',
                            'starttime':'ST',
                            'stoptime':'ET'}, inplace = True)

columns = ['S','E','ST','ET','B','U','BY','G','D']
data_ready = trip_data[columns]

In [ ]:
data_ready.head()

,S,E,ST,ET,B,U,BY,G,D
0,4065,3959,2020-12-01 00:00:03.7770,2020-12-01 00:11:38.3160,49454,Customer,1988,2,694
1,3284,3147,2020-12-01 00:00:09.1730,2020-12-01 01:09:00.2370,49524,Subscriber,1988,1,4131
2,3443,3563,2020-12-01 00:00:28.7470,2020-12-01 00:23:16.9980,34312,Subscriber,1977,2,1368
3,72,3336,2020-12-01 00:00:34.7680,2020-12-01 00:29:02.6470,45491,Subscriber,1982,0,1707
4,3165,3952,2020-12-01 00:00:36.4030,2020-12-01 00:36:59.0330,16558,Customer,1997,1,2182


In [ ]:
# Data velocity simulation
n = len(data_ready)
for i in range(0,n):

    trip = data_ready.iloc[[i]].to_dict(orient='records')
    
    producer.send(topic="citibike", value = trip[0])
    time.sleep(random.randint(0,5))